In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import numpy as np
import keras 
from numpy.lib.format import open_memmap

In [15]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# keras.backend.set_session(sess)

In [16]:
path_data = 'Extract_Data/'
train_X = np.load(path_data+'train_x.npy', mmap_mode='r')
train_Y = np.load(path_data+'train_y.npy', mmap_mode='r')
print(train_X.shape)

(912031, 20, 75)


In [17]:
test_X = np.load(path_data+'test_x.npy', mmap_mode='r')
test_Y = np.load(path_data+'test_y.npy', mmap_mode='r')
print(test_X.shape)

(371045, 20, 75)


In [18]:
def create_model(num_frame, num_joint, num_output):
    model = Sequential()
    model.add(CuDNNLSTM(50, input_shape=(num_frame, num_joint),return_sequences=True))
    model.add(Dropout(0.6))#使用Dropout函数可以使模型有更多的机会学习到多种独立的表征
    model.add(CuDNNLSTM(50, input_shape=(num_frame, num_joint)))
    model.add(Dropout(0.6))
    model.add(Dense(num_output, activation='softmax'))
    return model

In [19]:
load_model = True
max_frame = test_X.shape[1]
number_column = test_X.shape[2]
model = create_model(max_frame, number_column, 4)
start_epoch = 4

if load_model:
    weights_path = 'weights-improvement-04-0.75.hdf5'
    
    model.load_weights(weights_path)

sgd = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])    
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, )
callbacks_list = [checkpoint]

W0905 00:41:27.004787 139994104375104 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


In [20]:
model.fit(train_X, train_Y, epochs=200,
          validation_data=(test_X,test_Y), callbacks=callbacks_list, initial_epoch=start_epoch)

Train on 912031 samples, validate on 371045 samples
Epoch 5/200
912000/912031 [============================>.] - ETA: 0s - loss: 0.1714 - acc: 0.9400
Epoch 00005: val_acc improved from -inf to 0.54143, saving model to weights-improvement-05-0.54.hdf5
912031/912031 [==============================] - 8865s 10ms/sample - loss: 0.1714 - acc: 0.9400 - val_loss: 1.2212 - val_acc: 0.5414
Epoch 6/200
912000/912031 [============================>.] - ETA: 0s - loss: 0.1375 - acc: 0.9530
Epoch 00006: val_acc did not improve from 0.54143
912031/912031 [==============================] - 8353s 9ms/sample - loss: 0.1375 - acc: 0.9530 - val_loss: 1.3209 - val_acc: 0.5025
Epoch 7/200
912000/912031 [============================>.] - ETA: 0s - loss: 0.1165 - acc: 0.9608
Epoch 00007: val_acc improved from 0.54143 to 0.56866, saving model to weights-improvement-07-0.57.hdf5
912031/912031 [==============================] - 12558s 14ms/sample - loss: 0.1165 - acc: 0.9608 - val_loss: 1.2779 - val_acc: 0.5687


KeyboardInterrupt: 